In [ ]:
import os
import jCMIP as jc
Clist = jc.readList(('CMIP6listnew'))
from netCDF4 import Dataset
import numpy as np
from scipy.integrate import quad
import matplotlib.pyplot as plt
from scipy import stats as st
import cftime
import pandas as pd
%matplotlib inline

#### Historical

##### Write to file

In [ ]:
totmod =  ['CAMS-CSM1-0','CAS-ESM2-0','CESM2-WACCM','CIESM','CMCC-CM2-SR5','CMCC-ESM2','CNRM-CM6-1','CNRM-ESM2-1','CanESM5','EC-Earth3','FIO-ESM-2-0',
            'HadGEM3-GC31-LL','HadGEM3-GC31-MM','IPSL-CM6A-LR','MPI-ESM1-2-HR','MPI-ESM1-2-LR','MRI-ESM2-0','NESM3','UKESM1-0-LL']
ENS    =  ['r1i1p1f1','r1i1p1f1','r1i1p1f1','r1i1p1f1','r1i1p1f1','r1i1p1f1','r1i1p1f2','r2i1p1f2','r1i1p1f1','r1i1p1f1','r1i1p1f1','r1i1p1f3',
           'r1i1p1f3','r1i1p1f1','r1i1p1f1','r1i1p1f1','r1i1p1f1','r1i1p1f1','r1i1p1f2']
EXP    = 'historical'

for i in range(len(totmod)):
    print('Computing file ' + str(i+1) + ' of ' + str(len(totmod)))
    model   = Clist[totmod[i]]
    VFiles = model.getFiles('vo',    EXP=EXP,ENS=ENS[i],vtype='Omon',gtype='gn')
    WUFiles = model.getFiles('tauuo',EXP=EXP,ENS=ENS[i],vtype='Omon',gtype='gn')
    WVFiles  = model.getFiles('tauvo',EXP=EXP,ENS=ENS[i],vtype='Omon',gtype='gn') 
          
    if len(WUFiles) < 5:
        ncid = Dataset(WUFiles[-1],'r')
        tauuo = ncid.variables['tauuo'][:]
        ncid.close()
        
        ncid = Dataset(WVFiles[-1],'r')
        tauvo = ncid.variables['tauvo'][:]
        ncid.close() 
        
    if 8 < len(WUFiles) < 11:
        ncid = Dataset(WUFiles[-2],'r')
        tauuo = ncid.variables['tauuo'][:]
        ncid.close()
        
        ncid = Dataset(WUFiles[-1],'r')
        tauuo2 = ncid.variables['tauuo'][:]
        tauuo  = np.append(tauuo,tauuo2,axis=0)
        ncid.close()

        ncid = Dataset(WVFiles[-2],'r')
        tauvo = ncid.variables['tauvo'][:]
        ncid.close()
        #tauvo_mask = 1-tauvo[0,:,:].mask
        
        ncid = Dataset(WVFiles[-1],'r')
        tauvo2 = ncid.variables['tauvo'][:]
        tauvo  = np.append(tauvo,tauvo2,axis=0)
        ncid.close()
        
        tauuo = np.ma.masked_where(tauuo>1000,tauuo)     #Add mask info that is lost when using np.append
        tauvo = np.ma.masked_where(tauvo>1000,tauvo)
        
    if len(WUFiles) == 33:
        ncid = Dataset(WUFiles[-3],'r')
        tauuo = ncid.variables['tauuo'][:]
        ncid.close()

        ncid = Dataset(WVFiles[-3],'r')
        tauvo = ncid.variables['tauvo'][:]
        ncid.close()
               
        for k in range(2):
            ncid = Dataset(WUFiles[-2+k],'r')
            tauuo2 = ncid.variables['tauuo'][:]
            tauuo  = np.append(tauuo,tauuo2,axis=0)
            ncid.close() 

            ncid = Dataset(WVFiles[-2+k],'r')
            tauvo2 = ncid.variables['tauvo'][:]
            tauvo  = np.append(tauvo,tauvo2,axis=0)
            ncid.close()   

        tauuo = np.ma.masked_where(tauuo>1000,tauuo)    #Add mask info that is lost when using np.append
        tauvo = np.ma.masked_where(tauvo>1000,tauvo)


    if len(WUFiles) == 165:
        ncid = Dataset(WUFiles[-12],'r')
        tauuo = ncid.variables['tauuo'][:]
        ncid.close()
        
        ncid = Dataset(WVFiles[-12],'r')
        tauvo = ncid.variables['tauvo'][:]
        ncid.close()
            
        for k in range(11):
            ncid = Dataset(WUFiles[-11+k],'r')
            tauuo2 = ncid.variables['tauuo'][:]
            tauuo  = np.append(tauuo,tauuo2,axis=0)
            ncid.close() 

            ncid = Dataset(WVFiles[-11+k],'r')
            tauvo2 = ncid.variables['tauvo'][:]
            tauvo  = np.append(tauvo,tauvo2,axis=0)
            ncid.close()

        tauuo = np.ma.masked_where(tauuo>1000,tauuo)    #Add mask info that is lost when using np.append
        tauvo = np.ma.masked_where(tauvo>1000,tauvo)
        
    # Read in grid information:
    ncid = Dataset(VFiles[0],'r')
    if model.Oreg:
        lon  = ncid.variables[model.Olon][:]
        lat  = ncid.variables[model.Olat][:]
        lon,lat = np.meshgrid(lon,lat)
    else:
        lon  = ncid.variables[model.Olon][:,:]
        lat  = ncid.variables[model.Olat][:,:]
    ncid.close()
    
    mesh_mask = ('/home/users/jordi/mesh_masks/' + model.name + '_mesh_mask.nc')
    ncid = Dataset(mesh_mask,'r')
    vmask = np.squeeze(ncid.variables['vmask'][:,:,:,:])
    dx   = ncid.variables['dxv'][:,:]              
    dy   = ncid.variables['dyv'][:,:]   
    ncid.close()
    
    #Flip mask if necessary
    if model.OflipNS == True:
        mesh_mask = ('/home/users/jordi/mesh_masks/' + model.name + '_mesh_mask.nc')
        ncid = Dataset(mesh_mask,'r')
        vmask = np.flip(np.squeeze(ncid.variables['vmask'][:,:,:,:]),axis=1)
        dx   = np.flip(ncid.variables['dxv'][:,:],axis=0)              
        dy   = np.flip(ncid.variables['dyv'][:,:],axis=0)
        ncid.close()        
            
    #Find lat and lon indices
    alat = 26.5                           
    mlat = np.mean(lat,axis=1)
    ilat = np.argmin(np.abs(mlat-alat))
    
    lon[np.where(lon>180)] = lon[np.where(lon>180)] - 360
    alon_min = -80
    alon_max = -10
    ilon_min = np.argmin(np.abs(lon[ilat,:]-alon_min))
    ilon_max = np.argmin(np.abs(lon[ilat,:]-alon_max))

    #Shift Atlantic if necessary
    if model.name in ['CESM2-WACCM','CIESM','FIO-ESM-2-0']:
        tauuo = np.roll(tauuo[:,:,:],90,axis=-1)
        tauvo = np.roll(tauvo[:,:,:],90,axis=-1)
        vmask[:,:,:] = np.roll(vmask[:,:,:],90,axis=2)
        dx  = np.roll(dx,90,axis=-1)
        dy  = np.roll(dy,90,axis=-1)
        ilon_min = ilon_min + 90
        ilon_max = ilon_max + 90

        #Make sure the min and max fall within the mask longitudes
        if ilon_min > len(lon[0,:]):
            ilon_min = ilon_min - len(lon[0,:])
        elif ilon_min < 0:
                ilon_min = len(lon[0,:]) + ilon_min

        if ilon_max > len(lon[0,:]):
            ilon_max = ilon_max - len(lon[0,:])
        elif ilon_max < 0:
                ilon_max = len(lon[0,:]) + ilon_max 
        
    #Manual changes in ilon and ilat
    if model.name in ['CESM2-WACCM','FIO-ESM-2-0']:
        ilat = ilat + 3
        ilon_min = ilon_min - 1
    elif model.name == 'CIESM':
        ilat = ilat + 3
        ilon_min = ilon_min - 2
    elif model.name in ['CMCC-CM2-SR5','CMCC-ESM2','CNRM-CM6-1','CNRM-ESM2-1','EC-Earth3']:
        ilon_min = ilon_min - 1     
        ilon_max = ilon_max + 1       
    elif model.name in ['CanESM5','HadGEM3-GC31-LL','NESM3','UKESM1-0-LL']:
        ilon_min = ilon_min - 1        
    elif model.name == 'HadGEM3-GC31-MM':
        ilon_min = ilon_min - 2    
    elif model.name == 'IPSL-CM6A-LR':
        ilon_min = ilon_min - 1
        ilon_max = ilon_max - 1           
    elif model.name == 'MPI-ESM1-2-HR':
        ilat = ilat - 4
        ilon_min = ilon_min - 1
        ilon_max = ilon_max + 3           
    elif model.name == 'MPI-ESM1-2-LR':
        ilat = ilat - 8
        ilon_min = ilon_min - 3
        ilon_max = ilon_max - 1   
                
    #Correct for halo points
    if model.OextraWE == [1, 1]:
        vmask  = vmask[0,ilat,ilon_min-1:ilon_max-1]
        dx   = dx[ilat,ilon_min-1:ilon_max-1]
        dy   = dy[ilat,ilon_min-1:ilon_max-1]    
    else:
        vmask  = vmask[0,ilat,ilon_min:ilon_max]
        dx   = dx[ilat,ilon_min:ilon_max]
        dy   = dy[ilat,ilon_min:ilon_max]
    
    
    time = 129
    #Obtain all surrounding grid cells which are used for averaging
    tauuo_up       = tauuo[-time::,ilat+1,ilon_min:ilon_max]
    tauuo_down     = tauuo[-time::,ilat-1,ilon_min:ilon_max]
    tauuo_right    = np.roll(tauuo[-time::,ilat,:],-1,axis=-1)[:,ilon_min:ilon_max]
    tauuo_left     = np.roll(tauuo[-time::,ilat,:],1,axis=-1)[:,ilon_min:ilon_max]
    tauuo_upleft   = np.roll(tauuo[-time::,ilat+1,:],1,axis=-1)[:,ilon_min:ilon_max]
    tauuo_downleft = np.roll(tauuo[-time::,ilat-1,:],1,axis=-1)[:,ilon_min:ilon_max]
    tauuo          = tauuo[-time::,ilat,ilon_min:ilon_max]

    tauvo_up     = tauvo[-time::,ilat+1,ilon_min:ilon_max]
    tauvo_down   = tauvo[-time::,ilat-1,ilon_min:ilon_max]
    tauvo_right  = np.roll(tauvo[-time::,ilat,:],-1,axis=-1)[:,ilon_min:ilon_max]
    tauvo_left   = np.roll(tauvo[-time::,ilat,:],1,axis=-1)[:,ilon_min:ilon_max]
    tauvo        = tauvo[-time::,ilat,ilon_min:ilon_max]
    
    
    #All B-grids have the same averaging, C-grids vary
    if model.Ogrid in ['Btr','Bbl']:
        du = (tauuo_up + tauuo)/2 - (tauuo_down + tauuo)/2
        dv = (tauvo_right + tauvo)/2 - (tauvo_left + tauvo)/2       
        wsc = dv/dx[None,:] - du/dy[None,:]                      #None is included to divide a 2D array by a 1D array
    elif model.Ogrid == 'Ctr':
        du = (tauuo_up + tauuo_upleft)/2 - (tauuo + tauuo_left)/2
        dv = (tauvo_right + tauvo)/2 - (tauvo_left + tauvo)/2        
        wsc = dv/dx[None,:] - du/dy[None,:] 
    elif model.Ogrid == 'Cbr':
        du = (tauuo + tauuo_left)/2 - (tauuo_down + tauuo_downleft)/2
        dv = (tauvo_right + tauvo)/2 - (tauvo_left + tauvo)/2       
        wsc = dv/dx[None,:] - du/dy[None,:] 
    
    #Write to file
    outfile = ('/home/users/jordi/wind_stress_curl/' + totmod[i] + '_' + EXP + '_' + ENS[i] + '.nc')
    ncid = Dataset(outfile,'w')
    ncid.createDimension('time', time)
    ncid.createDimension('lon' , ilon_max-ilon_min)
    
    ncid.createVariable('wsc10','f8',('time','lon',))
    ncid.variables['wsc10'][:]       = wsc
    ncid.close()

##### Plot results

In [ ]:
totmod =  ['CAMS-CSM1-0','CAS-ESM2-0','CESM2-WACCM','CIESM','CMCC-CM2-SR5','CMCC-ESM2','CNRM-CM6-1','CNRM-ESM2-1','CanESM5','EC-Earth3','FIO-ESM-2-0',
            'HadGEM3-GC31-LL','HadGEM3-GC31-MM','IPSL-CM6A-LR','MPI-ESM1-2-HR','MPI-ESM1-2-LR','MRI-ESM2-0','NESM3','UKESM1-0-LL']
ENS    =  ['r1i1p1f1','r1i1p1f1','r1i1p1f1','r1i1p1f1','r1i1p1f1','r1i1p1f1','r1i1p1f2','r2i1p1f2','r1i1p1f1','r1i1p1f1','r1i1p1f1','r1i1p1f3',
           'r1i1p1f3','r1i1p1f1','r1i1p1f1','r1i1p1f1','r1i1p1f1','r1i1p1f1','r1i1p1f2']
EXP    = 'historical'

#Change time to date
outfile = ('/home/users/jordi/section26data/' + totmod[6] + '_' + EXP + '_' + ENS[6] + '.nc')  #CNRM-CM6-1 has gregorian calendar, which is supported by python datetime objects
ncid    = Dataset(outfile,'r')
time    = ncid.variables['time'][-129::]
units   = ncid.variables['time'].units
cal     = ncid.variables['time'].calendar
ncid.close()

tdate_hist   = cftime.num2pydate(time,units)

plt.figure(1,figsize=(10,7))
plt.gca().set_prop_cycle(color=['tab:blue','tab:orange','tab:green','tab:red','tab:purple','tab:brown','tab:pink','tab:gray','tab:olive','tab:cyan','maroon','sienna','goldenrod','greenyellow','limegreen','mediumslateblue','pink','tomato','lightgreen'])
std = np.zeros(len(totmod))
wsc_histmean = np.zeros((len(totmod),129))
for i in range(len(totmod)):    
    outfile = ('/home/users/jordi/wind_stress_curl/' + totmod[i] + '_' + EXP + '_' + ENS[i] + '.nc')
    ncid    = Dataset(outfile,'r')
    wsc10   = ncid.variables['wsc10'][:,:]
    ncid.close()
    
    outfile1 = ('/home/users/jordi/section26data/' + totmod[i] + '_' + EXP + '_' + ENS[i] + '.nc')
    ncid = Dataset(outfile1,'r')
    lon  = np.squeeze(ncid.variables['lon'][:,:])
    ncid.close()
    
    #Correct lon for Atlantic shift
    if totmod[i] in ['CESM2-WACCM','CIESM','FIO-ESM-2-0']:
        lon = lon - 100
    
    #Find first and last unmasked grid cells
    x = np.nanmean(wsc10,axis=0)
    x1 = np.nonzero(x)
    
    plt.plot(lon[(np.min(x1)+1):(np.max(x1)-1)],np.nanmean(wsc10[:,(np.min(x1)+1):(np.max(x1)-1)],axis=0),label=totmod[i])  
    
    wsc_histmean[i,:] = np.nanmean(wsc10[:,(np.min(x1)+1):(np.max(x1)-1)],axis=1)
    std[i] = np.std(np.nanmean(wsc10[:,(np.min(x1)+1):(np.max(x1)-1)],axis=0))
    print(str(totmod[i]) + '   ' + str(std[i]))
          
plt.legend(loc='center left',bbox_to_anchor=(1.02, 0.5),prop={'size': 12},frameon=False)
print('Ensemble mean std = ' + str(np.nanmean(std)) )

#### SSP585

##### Write to file

In [ ]:
totmod =  ['CAMS-CSM1-0','CAS-ESM2-0','CESM2-WACCM','CIESM','CMCC-CM2-SR5','CMCC-ESM2','CNRM-CM6-1','CanESM5','EC-Earth3','FIO-ESM-2-0',
            'HadGEM3-GC31-LL','HadGEM3-GC31-MM','IPSL-CM6A-LR','MPI-ESM1-2-HR','MPI-ESM1-2-LR','MRI-ESM2-0','NESM3','UKESM1-0-LL']
ENS    =  ['r1i1p1f1','r1i1p1f1','r1i1p1f1','r1i1p1f1','r1i1p1f1','r1i1p1f1','r1i1p1f2','r1i1p1f1','r1i1p1f1','r1i1p1f1','r1i1p1f3',
           'r1i1p1f3','r1i1p1f1','r1i1p1f1','r1i1p1f1','r1i1p1f1','r1i1p1f1','r1i1p1f2']
EXP    = 'ssp585'

for i in range(len(totmod)):
    print('Computing file ' + str(i+1) + ' of ' + str(len(totmod)))
    model   = Clist[totmod[i]]
    VFiles = model.getFiles('vo',    EXP=EXP,ENS=ENS[i],vtype='Omon',gtype='gn')
    WUFiles = model.getFiles('tauuo',EXP=EXP,ENS=ENS[i],vtype='Omon',gtype='gn')
    WVFiles  = model.getFiles('tauvo',EXP=EXP,ENS=ENS[i],vtype='Omon',gtype='gn') 
          
    ncid = Dataset(WUFiles[0],'r')
    tauuo = ncid.variables['tauuo'][:]
    ncid.close()
        
    ncid = Dataset(WVFiles[0],'r')
    tauvo = ncid.variables['tauvo'][:]
    ncid.close()    
    
    if totmod[i] in ['CIESM','HadGEM3-GC31-LL','UKESM1-0-LL']:
        ncid = Dataset(WUFiles[1],'r')
        tauuo2 = ncid.variables['tauuo'][:]
        ncid.close()
        tauuo  = np.append(tauuo,tauuo2,axis=0)        
        
        ncid = Dataset(WVFiles[1],'r')
        tauvo2 = ncid.variables['tauvo'][:]
        ncid.close() 
        tauvo  = np.append(tauvo,tauvo2,axis=0)
        
        tauuo = np.ma.masked_where(tauuo>1000,tauuo)
        tauvo = np.ma.masked_where(tauvo>1000,tauvo)
        
    if totmod[i] in ['HadGEM3-GC31-MM','MPI-ESM1-2-LR']:
        for k in range(4):
            ncid = Dataset(WUFiles[k+1],'r')
            tauuo2 = ncid.variables['tauuo'][:]
            tauuo  = np.append(tauuo,tauuo2,axis=0)
            ncid.close() 

            ncid = Dataset(WVFiles[k+1],'r')
            tauvo2 = ncid.variables['tauvo'][:]
            tauvo  = np.append(tauvo,tauvo2,axis=0)
            ncid.close()   
               
        tauuo = np.ma.masked_where(tauuo>1000,tauuo)
        tauvo = np.ma.masked_where(tauvo>1000,tauvo)
        
    if totmod[i] == 'MPI-ESM1-2-HR':
        for k in range(17):
            ncid = Dataset(WUFiles[k+1],'r')
            tauuo2 = ncid.variables['tauuo'][:]
            tauuo  = np.append(tauuo,tauuo2,axis=0)
            ncid.close() 

            ncid = Dataset(WVFiles[k+1],'r')
            tauvo2 = ncid.variables['tauvo'][:]
            tauvo  = np.append(tauvo,tauvo2,axis=0)
            ncid.close()   
               
        tauuo = np.ma.masked_where(tauuo>1000,tauuo)
        tauvo = np.ma.masked_where(tauvo>1000,tauvo)

    if totmod[i] == 'EC-Earth3':
        for k in range(85):
            ncid = Dataset(WUFiles[k+1],'r')
            tauuo2 = ncid.variables['tauuo'][:]
            tauuo  = np.append(tauuo,tauuo2,axis=0)
            ncid.close() 

            ncid = Dataset(WVFiles[k+1],'r')
            tauvo2 = ncid.variables['tauvo'][:]
            tauvo  = np.append(tauvo,tauvo2,axis=0)
            ncid.close()   
               
        tauuo = np.ma.masked_where(tauuo>1000,tauuo)
        tauvo = np.ma.masked_where(tauvo>1000,tauvo)
        
    # Read in grid information:
    ncid = Dataset(VFiles[0],'r')
    if model.Oreg:
        lon  = ncid.variables[model.Olon][:]
        lat  = ncid.variables[model.Olat][:]
        lon,lat = np.meshgrid(lon,lat)
    else:
        lon  = ncid.variables[model.Olon][:,:]
        lat  = ncid.variables[model.Olat][:,:]
    ncid.close()
    
    mesh_mask = ('/home/users/jordi/mesh_masks/' + model.name + '_mesh_mask.nc')
    ncid = Dataset(mesh_mask,'r')
    vmask = np.squeeze(ncid.variables['vmask'][:,:,:,:])
    dx   = ncid.variables['dxv'][:,:]              
    dy   = ncid.variables['dyv'][:,:]   
    ncid.close()
    
    if model.OflipNS == True:
        mesh_mask = ('/home/users/jordi/mesh_masks/' + model.name + '_mesh_mask.nc')
        ncid = Dataset(mesh_mask,'r')
        vmask = np.flip(np.squeeze(ncid.variables['vmask'][:,:,:,:]),axis=1)
        dx   = np.flip(ncid.variables['dxv'][:,:],axis=0)              
        dy   = np.flip(ncid.variables['dyv'][:,:],axis=0)
        ncid.close()        
            
    #Find lat and lon indices
    alat = 26.5                           
    mlat = np.mean(lat,axis=1)
    ilat = np.argmin(np.abs(mlat-alat))
    
    lon[np.where(lon>180)] = lon[np.where(lon>180)] - 360
    alon_min = -80
    alon_max = -10
    ilon_min = np.argmin(np.abs(lon[ilat,:]-alon_min))
    ilon_max = np.argmin(np.abs(lon[ilat,:]-alon_max))

    #Shift Atlantic if necessary
    if model.name in ['CESM2-WACCM','CIESM','FIO-ESM-2-0']:
        tauuo = np.roll(tauuo[:,:,:],90,axis=-1)
        tauvo = np.roll(tauvo[:,:,:],90,axis=-1)
        vmask[:,:,:] = np.roll(vmask[:,:,:],90,axis=2)
        dx  = np.roll(dx,90,axis=-1)
        dy  = np.roll(dy,90,axis=-1)
        ilon_min = ilon_min + 90
        ilon_max = ilon_max + 90

        #Make sure the min and max fall within the mask longitudes
        if ilon_min > len(lon[0,:]):
            ilon_min = ilon_min - len(lon[0,:])
        elif ilon_min < 0:
                ilon_min = len(lon[0,:]) + ilon_min

        if ilon_max > len(lon[0,:]):
            ilon_max = ilon_max - len(lon[0,:])
        elif ilon_max < 0:
                ilon_max = len(lon[0,:]) + ilon_max 
        
    #Manual change in ilon and ilat
    if model.name in ['CESM2-WACCM','FIO-ESM-2-0']:
        ilat = ilat + 3
        ilon_min = ilon_min - 1
    elif model.name == 'CIESM':
        ilat = ilat + 3
        ilon_min = ilon_min - 2
    elif model.name in ['CMCC-CM2-SR5','CMCC-ESM2','CNRM-CM6-1','CNRM-ESM2-1','EC-Earth3']:
        ilon_min = ilon_min - 1     
        ilon_max = ilon_max + 1       
    elif model.name in ['CanESM5','HadGEM3-GC31-LL','NESM3','UKESM1-0-LL']:
        ilon_min = ilon_min - 1        
    elif model.name == 'HadGEM3-GC31-MM':
        ilon_min = ilon_min - 2    
    elif model.name == 'IPSL-CM6A-LR':
        ilon_min = ilon_min - 1
        ilon_max = ilon_max - 1           
    elif model.name == 'MPI-ESM1-2-HR':
        ilat = ilat - 4
        ilon_min = ilon_min - 1
        ilon_max = ilon_max + 3           
    elif model.name == 'MPI-ESM1-2-LR':
        ilat = ilat - 8
        ilon_min = ilon_min - 3
        ilon_max = ilon_max - 1   
        
    #plt.figure(i,figsize=(10,7))
    #plt.title(totmod[i])
    #if model.OextraWE == [1, 1]:
    #    plt.pcolormesh(1-tauuo[0,:,1:-1].mask+2*vmask[0,:,:])
    #else:
    #    plt.pcolormesh(1-tauuo[0,:,:].mask+2*vmask[0,:,:])
                
    #Correct for halo points
    if model.OextraWE == [1, 1]:
        vmask  = vmask[0,ilat,ilon_min-1:ilon_max-1]
        dx   = dx[ilat,ilon_min-1:ilon_max-1]
        dy   = dy[ilat,ilon_min-1:ilon_max-1]    
    else:
        vmask  = vmask[0,ilat,ilon_min:ilon_max]
        dx   = dx[ilat,ilon_min:ilon_max]
        dy   = dy[ilat,ilon_min:ilon_max]
    
    #plt.figure(i)
    #plt.title(totmod[i])
    #plt.pcolormesh(tauuo[-1,:,:])
    #plt.axhline(y=ilat,color='C3')
    #plt.axvline(x=ilon_min,color='C2')
    #plt.axvline(x=ilon_max,color='C0')
    #plt.colorbar()

    #plt.figure(i+1)
    #plt.title(totmod[i])
    #plt.pcolormesh(tauvo[-1,:,:])
    #plt.axhline(y=ilat,color='C3')
    #plt.axvline(x=ilon_min,color='C2')
    #plt.axvline(x=ilon_max,color='C0')
    #plt.colorbar()
    
    time = 85*12
    #Obtain all surrounding grid cells which are used for averaging
    tauuo_up       = tauuo[0:time,ilat+1,ilon_min:ilon_max]
    tauuo_down     = tauuo[0:time,ilat-1,ilon_min:ilon_max]
    tauuo_right    = np.roll(tauuo[0:time,ilat,:],-1,axis=-1)[:,ilon_min:ilon_max]
    tauuo_left     = np.roll(tauuo[0:time,ilat,:],1,axis=-1)[:,ilon_min:ilon_max]
    tauuo_upleft   = np.roll(tauuo[0:time,ilat+1,:],1,axis=-1)[:,ilon_min:ilon_max]
    tauuo_downleft = np.roll(tauuo[0:time,ilat-1,:],1,axis=-1)[:,ilon_min:ilon_max]
    tauuo          = tauuo[0:time,ilat,ilon_min:ilon_max]

    tauvo_up     = tauvo[0:time,ilat+1,ilon_min:ilon_max]
    tauvo_down   = tauvo[0:time,ilat-1,ilon_min:ilon_max]
    tauvo_right  = np.roll(tauvo[0:time,ilat,:],-1,axis=-1)[:,ilon_min:ilon_max]
    tauvo_left   = np.roll(tauvo[0:time,ilat,:],1,axis=-1)[:,ilon_min:ilon_max]
    tauvo        = tauvo[0:time,ilat,ilon_min:ilon_max]
    
    #plt.figure(i+2)
    #plt.title(totmod[i])
    #plt.pcolormesh(tauuo)
    #plt.colorbar()
    
    #plt.figure()
    #plt.title(totmod[i])
    #plt.pcolormesh(tauvo)
    #plt.colorbar()
    
    #All B-grids have the same averaging, C-grids vary
    if model.Ogrid in ['Btr','Bbl']:
        du = (tauuo_up + tauuo)/2 - (tauuo_down + tauuo)/2
        dv = (tauvo_right + tauvo)/2 - (tauvo_left + tauvo)/2       
        wsc = dv/dx[None,:] - du/dy[None,:]                            #None is included to divide a 2D array by a 1D array
    elif model.Ogrid == 'Ctr':
        du = (tauuo_up + tauuo_upleft)/2 - (tauuo + tauuo_left)/2
        dv = (tauvo_right + tauvo)/2 - (tauvo_left + tauvo)/2        
        wsc = dv/dx[None,:] - du/dy[None,:] 
    elif model.Ogrid == 'Cbr':
        du = (tauuo + tauuo_left)/2 - (tauuo_down + tauuo_downleft)/2
        dv = (tauvo_right + tauvo)/2 - (tauvo_left + tauvo)/2       
        wsc = dv/dx[None,:] - du/dy[None,:]  
    
    #Write to file
    outfile = ('/home/users/jordi/wind_stress_curl/' + totmod[i] + '_' + EXP + '_' + ENS[i] + '.nc')
    ncid = Dataset(outfile,'w')
    ncid.createDimension('time', time)
    ncid.createDimension('lon' , ilon_max-ilon_min)
    
    ncid.createVariable('wsc10','f8',('time','lon',))
    ncid.variables['wsc10'][:]       = wsc
    ncid.close()

##### Plot results

In [ ]:
totmod =  ['CAMS-CSM1-0','CAS-ESM2-0','CESM2-WACCM','CIESM','CMCC-CM2-SR5','CMCC-ESM2','CNRM-CM6-1','CanESM5','EC-Earth3','FIO-ESM-2-0',
            'HadGEM3-GC31-LL','HadGEM3-GC31-MM','IPSL-CM6A-LR','MPI-ESM1-2-HR','MPI-ESM1-2-LR','MRI-ESM2-0','NESM3','UKESM1-0-LL']
ENS    =  ['r1i1p1f1','r1i1p1f1','r1i1p1f1','r1i1p1f1','r1i1p1f1','r1i1p1f1','r1i1p1f2','r1i1p1f1','r1i1p1f1','r1i1p1f1','r1i1p1f3',
           'r1i1p1f3','r1i1p1f1','r1i1p1f1','r1i1p1f1','r1i1p1f1','r1i1p1f1','r1i1p1f2']
EXP    = 'ssp585'

#Change time to date
outfile = ('/home/users/jordi/section26data/' + totmod[0] + '_' + EXP + '_' + ENS[0] + '.nc')   #First model data: 2015-01-16 - 2099-12-16
ncid    = Dataset(outfile,'r')
time    = ncid.variables['time'][:]
ncid.close()

tdate_ssp = pd.to_datetime(time,unit='D',origin=pd.Timestamp('2015-01-16'))

plt.figure(1,figsize=(10,7))
plt.gca().set_prop_cycle(color=['tab:blue','tab:orange','tab:green','tab:red','tab:purple','tab:brown','tab:pink','tab:gray','tab:olive','tab:cyan','maroon','sienna','goldenrod','greenyellow','limegreen','mediumslateblue','pink','tomato','lightgreen'])
std = np.zeros(len(totmod))
wsc_sspmean = np.zeros((len(totmod),85*12))
for i in range(len(totmod)):    
    outfile = ('/home/users/jordi/wind_stress_curl/' + totmod[i] + '_' + EXP + '_' + ENS[i] + '.nc')
    ncid    = Dataset(outfile,'r')
    wsc10   = ncid.variables['wsc10'][:,:]
    ncid.close()
    
    outfile1 = ('/home/users/jordi/section26data/' + totmod[i] + '_' + EXP + '_' + ENS[i] + '.nc')
    ncid = Dataset(outfile1,'r')
    lon  = np.squeeze(ncid.variables['lon'][:,:])
    ncid.close()
    
    #Correct lon for Atlantic shift
    if totmod[i] in ['CESM2-WACCM','CIESM','FIO-ESM-2-0']:
        lon = lon - 100
    
    #Find first and last unmasked grid cells
    x = np.nanmean(wsc10,axis=0)
    x1 = np.nonzero(x)
    
    plt.plot(lon[(np.min(x1)+1):(np.max(x1)-1)],np.nanmean(wsc10[:,(np.min(x1)+1):(np.max(x1)-1)],axis=0),label=totmod[i])  
    
    wsc_sspmean[i,:] = np.nanmean(wsc10[:,(np.min(x1)+1):(np.max(x1)-1)],axis=1)
    std[i] = np.std(np.nanmean(wsc10[:,(np.min(x1)+1):(np.max(x1)-1)],axis=0))
    
    print(str(totmod[i]) + '   ' + str(std[i]))
          
plt.legend(loc='center left',bbox_to_anchor=(1.02, 0.5),prop={'size': 12},frameon=False)
print('Ensemble mean std = ' + str(np.nanmean(std)) )

#### Compare historical and ssp585 ensemble mean time series

In [ ]:
print('Historical     ' +  str(np.nanmean(wsc_histmean)))
print('SSP585         ' + str(np.nanmean(wsc_sspmean)))
print('SSP585 90-99   ' + str(np.nanmean(wsc_sspmean[:,-10*12::])))

#Compute decline of last decade of SSP585 compared to historical mean
dcl = 100*(np.nanmean(wsc_sspmean[:,-10*12::]) - np.nanmean(wsc_histmean))/np.nanmean(wsc_histmean)
print(dcl)

#Plot historical ensemble mean time series
fig,ax = plt.subplots(figsize=(10,7))
plt.plot(tdate_hist,np.nanmean(wsc_histmean,axis=0))
plt.ylabel('Wind stress curl (s^-1)',fontsize=14)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.xlim(left=tdate_hist[0])
ax.yaxis.get_offset_text().set_fontsize(14)

#Plot SSP585 ensemble mean time series
fig,ax = plt.subplots(figsize=(10,7))
plt.plot(tdate_ssp,np.nanmean(wsc_sspmean,axis=0))
plt.ylabel('Wind stress curl (s^-1)',fontsize=16)
plt.xticks(fontsize=16)
plt.yticks(fontsize=16)
plt.text(tdate_ssp[-1],np.nanmean(wsc_sspmean[:,-10*12::]),' ' + str(int(dcl)) + '%',fontsize=16)    #Add decline to graph
plt.xlim(left=tdate_ssp[0],right=pd.Timestamp('2108-01-16'))
ax.yaxis.get_offset_text().set_fontsize(16)

#### Check length wind stress files

In [ ]:
totmod =  ['CAMS-CSM1-0','CAS-ESM2-0','CESM2-WACCM','CIESM','CMCC-CM2-SR5','CMCC-ESM2','CNRM-CM6-1','CanESM5','EC-Earth3','FIO-ESM-2-0',
            'HadGEM3-GC31-LL','HadGEM3-GC31-MM','IPSL-CM6A-LR','MPI-ESM1-2-HR','MPI-ESM1-2-LR','MRI-ESM2-0','NESM3','UKESM1-0-LL']
ENS    =  ['r1i1p1f1','r1i1p1f1','r1i1p1f1','r1i1p1f1','r1i1p1f1','r1i1p1f1','r1i1p1f2','r1i1p1f1','r1i1p1f1','r1i1p1f1','r1i1p1f3',
           'r1i1p1f3','r1i1p1f1','r1i1p1f1','r1i1p1f1','r1i1p1f1','r1i1p1f1','r1i1p1f2']
ENSar  = ['r2i1p1f1','r1i1p1f1','r1i1p1f1','r1i1p1f1','r1i1p1f1','r1i1p1f1','r1i1p1f2','r2i1p1f2','r1i1p1f1','r1i1p1f1','r1i1p1f1','r1i1p1f1',
           'r1i1p1f1','r1i1p1f1','r1i1p1f1','r1i1p1f1','r1i1p1f1','r1i1p1f1','r1i1p1f2']
EXP    = 'ssp585'

for i in range(len(totmod)):
    model   = Clist[totmod[i]]
    WUFiles = model.getFiles('tauuo',EXP=EXP,ENS=ENS[i],vtype='Omon',gtype='gn')
    print(str(totmod[i]) + ' - ' + str(len(WUFiles)))
    WVFiles  = model.getFiles('tauvo',EXP=EXP,ENS=ENS[i],vtype='Omon',gtype='gn')
    print(str(totmod[i]) + ' - ' + str(np.shape(WVFiles)))   
    
    if model.name in ['CESM2-WACCM','CIESM','CanESM5','EC-Earth3','HadGEM3-GC31-LL','HadGEM3-GC31-MM','IPSL-CM6A-LR','MPI-ESM1-2-HR','MPI-ESM1-2-LR','MRI-ESM2-0','UKESM1-0-LL']:
        print(WUFiles[-1])